In [1]:
import bs4
from bs4 import BeautifulSoup
import requests as rq
import re
import pandas as pd
import numpy as np
import datetime 
import os
import math
import pymongo

In [2]:
def convdollar(x):
    """
    Just a parsing function converting 2.5k to 2500, 1mil to 1000000
    """
    if 'k' in x:
        return float(x.replace('k',''))*1000
    else:
        return float(x)*1000000

In [3]:
millnames = ['',' Thousand',' Million',' Billion',' Trillion']

def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

In [4]:
def scrape():
    """
    Gets all box office data from 2018 to present from boxofficemojo.com
    """
    years=[str(a) for a in range(2018,2019)]
    df_list=[]
    for year in years:
        r=rq.get('https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=%s&p=.htm' % year)
        print('Box Office data for %s scraped' % year)
        p=BeautifulSoup(r.text,'html.parser')
            ### Look for the table ### 
        b=p.find_all('table')
        
        ### Usually the fourth table object on page ### 
        tb=b[3].find_all('td')
         ## Each data field is found in a <td> element in the fourth table. Store all data in a list ## 
        data=[]
        for i in tb:
            if i.find('a')!=None:
                data.append(i.find('a').contents[0])
            elif i.find('font')!=None:
                 data.append(i.find('font').contents[0])
            elif i.find('b')!=None:
                data.append(i.find('b').contents[0])
                ### Still a <b> tag left for <font> tags ## 
        data=[a.contents[0] if type(a)!=bs4.element.NavigableString else a for a in data]
        
        ### Strip special characters ### 
        data=[re.sub('[^A-Za-z0-9-. ]+', '', a) for a in data]
        
        ### Fill NaNs ### 
        data=[np.nan if a =='na' else a for a in data]
        
        ### Define the feature names ###
        columns=['bo_year_rank','title','studio','worldwide-gross','domestic-gross','domestic-pct','overseas-gross','overseas-pct']
        
        ### First 6 elements are column headers # 
        to_df=data[6:]
        
        ### Escape clause in case the layout changes from year to year ### 
        if len(to_df)%len(columns) != 0:
            print('Possible table misalignment in table for year %s' % year)
            break
            
            ### Convert to pandas dataframe ### 
        
        nrow=int(len(to_df)/len(columns))
        df=pd.DataFrame(np.array(to_df).reshape(nrow,8),columns=columns)
        df[['worldwide-gross','domestic-gross','overseas-gross']]=df[['worldwide-gross','domestic-gross','overseas-gross']].applymap(lambda x:convdollar(x))
        df['bo_year']=int(year)
        df_list.append(df)

        main=pd.concat(df_list)
    
        # Store data into csv # 
        #main.to_csv(os.path.join("output","current_boxoffice_mojo.csv"))
        return (main)
        print (main)
  

In [5]:
if __name__ == "__main__": 

    dirtymovies_df=scrape()

Box Office data for 2018 scraped


In [6]:
 dirtymovies_df

bo_year_rank                                              title  \
0              1                              Avengers Infinity War   
1              2                                      Black Panther   
2              3                      Jurassic World Fallen Kingdom   
3              4                                      Incredibles 2   
4              5                                            Aquaman   
5              6                                  Bohemian Rhapsody   
6              7                                         Venom 2018   
7              8                       Mission Impossible - Fallout   
8              9                                         Deadpool 2   
9             10         Fantastic Beasts The Crimes of Grindelwald   
10            11                               Ant-Man and the Wasp   
11            12                                   Ready Player One   
12            13                                  Operation Red Sea   
13            14                              Detective Chinatown 2   
14            15                                            The Meg   
15            16                          Ralph Breaks the Internet   
16            17               Hotel Transylvania 3 Summer Vacation   
17            18                          Dr. Seuss The Grinch 2018   
18            19                                          Bumblebee   
19            20                                A Star is Born 2018   
20            21                                       Rampage 2018   
21            22                         Mamma Mia Here We Go Again   
22            23                             Solo A Star Wars Story   
23            24                   Spider-Man Into The Spider-Verse   
24            25                                 Fifty Shades Freed   
25            26                                            The Nun   
26            27                                     Monster Hunt 2   
27            28                                       Peter Rabbit   
28            29                               Mary Poppins Returns   
29            30                                      A Quiet Place   
..           ...                                                ...   
275          276                                   I Am Not a Witch   
276          277                                Racer  the Jailbird   
277          278                                              Rodin   
278          279                               What Will People Say   
279          280                                          A Ciambra   
280          281                                   Oolong Courtyard   
281          282                                 Girls vs Gangsters   
282          283                               No Date No Signature   
283          284                                The Apparition 2018   
284          285                                        The Charmer   
285          286                       Elliot The Littlest Reindeer   
286          287                                       The Workshop   
287          288                                       Loving Pablo   
288          289                                  A Paris Education   
289          290                           Tag Along The Devil Fish   
290          291                           Gabriel and the Mountain   
291          292                  Marlina the Murderer in Four Acts   
292          293                                         The Escape   
293          294  How Victor The Garlic Took Alexey The Stud to ...   
294          295          Lou Andreas-Salom the Audacity to be Free   
295          296                                    Let Yourself Go   
296          297                                        Hannah 2018   
297          298                                           Souvenir   
298          299                         Furious Legend of Kolovrat   
299          300            

In [7]:
## Cleaning data by dropping the unessery rows (10-87) drops the bottom 78 trows
dirtymovies_df[:-295]


bo_year_rank                                       title studio  \
0            1                       Avengers Infinity War     BV   
1            2                               Black Panther     BV   
2            3               Jurassic World Fallen Kingdom   Uni.   
3            4                               Incredibles 2     BV   
4            5                                     Aquaman     WB   
5            6                           Bohemian Rhapsody    Fox   
6            7                                  Venom 2018   Sony   
7            8                Mission Impossible - Fallout   Par.   
8            9                                  Deadpool 2    Fox   
9           10  Fantastic Beasts The Crimes of Grindelwald     WB   

   worldwide-gross  domestic-gross domestic-pct  overseas-gross overseas-pct  \
0     2.048400e+09     678800000.0         33.1    1.369500e+09         66.9   
1     1.346900e+09     700100000.0         52.0    6.469000e+08         48.0   
2     1.309500e+09     417700000.0         31.9    8.918000e+08         68.1   
3     1.242800e+09     608600000.0         49.0    6.342000e+08         51.0   
4     1.147700e+09     335100000.0         29.2    8.126000e+08         70.8   
5     9.026000e+08     216400000.0         24.0    6.862000e+08         76.0   
6     8.550000e+08     213500000.0         25.0    6.415000e+08         75.0   
7     7.911000e+08     220200000.0         27.8    5.710000e+08         72.2   
8     7.789000e+08     318500000.0         40.9    4.605000e+08         59.1   
9     6.537000e+08     159600000.0         24.4    4.941000e+08         75.6   

   bo_year  
0     2018  
1     2018  
2     2018  
3     2018  
4     2018  
5     2018  
6     2018  
7     2018  
8     2018  
9     2018

In [8]:
#create new dataframe for top 10 
toptenmovies_df = dirtymovies_df[:-295]


In [9]:
toptenmovies_df

bo_year_rank                                       title studio  \
0            1                       Avengers Infinity War     BV   
1            2                               Black Panther     BV   
2            3               Jurassic World Fallen Kingdom   Uni.   
3            4                               Incredibles 2     BV   
4            5                                     Aquaman     WB   
5            6                           Bohemian Rhapsody    Fox   
6            7                                  Venom 2018   Sony   
7            8                Mission Impossible - Fallout   Par.   
8            9                                  Deadpool 2    Fox   
9           10  Fantastic Beasts The Crimes of Grindelwald     WB   

   worldwide-gross  domestic-gross domestic-pct  overseas-gross overseas-pct  \
0     2.048400e+09     678800000.0         33.1    1.369500e+09         66.9   
1     1.346900e+09     700100000.0         52.0    6.469000e+08         48.0   
2     1.309500e+09     417700000.0         31.9    8.918000e+08         68.1   
3     1.242800e+09     608600000.0         49.0    6.342000e+08         51.0   
4     1.147700e+09     335100000.0         29.2    8.126000e+08         70.8   
5     9.026000e+08     216400000.0         24.0    6.862000e+08         76.0   
6     8.550000e+08     213500000.0         25.0    6.415000e+08         75.0   
7     7.911000e+08     220200000.0         27.8    5.710000e+08         72.2   
8     7.789000e+08     318500000.0         40.9    4.605000e+08         59.1   
9     6.537000e+08     159600000.0         24.4    4.941000e+08         75.6   

   bo_year  
0     2018  
1     2018  
2     2018  
3     2018  
4     2018  
5     2018  
6     2018  
7     2018  
8     2018  
9     2018

In [10]:
##clean new dataframe for top 10 by removing rows domestic-pct, overseas-pct
toptenmovies_df.drop("domestic-pct", axis=1).drop("overseas-pct", axis=1)


bo_year_rank                                       title studio  \
0            1                       Avengers Infinity War     BV   
1            2                               Black Panther     BV   
2            3               Jurassic World Fallen Kingdom   Uni.   
3            4                               Incredibles 2     BV   
4            5                                     Aquaman     WB   
5            6                           Bohemian Rhapsody    Fox   
6            7                                  Venom 2018   Sony   
7            8                Mission Impossible - Fallout   Par.   
8            9                                  Deadpool 2    Fox   
9           10  Fantastic Beasts The Crimes of Grindelwald     WB   

   worldwide-gross  domestic-gross  overseas-gross  bo_year  
0     2.048400e+09     678800000.0    1.369500e+09     2018  
1     1.346900e+09     700100000.0    6.469000e+08     2018  
2     1.309500e+09     417700000.0    8.918000e+08     2018  
3     1.242800e+09     608600000.0    6.342000e+08     2018  
4     1.147700e+09     335100000.0    8.126000e+08     2018  
5     9.026000e+08     216400000.0    6.862000e+08     2018  
6     8.550000e+08     213500000.0    6.415000e+08     2018  
7     7.911000e+08     220200000.0    5.710000e+08     2018  
8     7.789000e+08     318500000.0    4.605000e+08     2018  
9     6.537000e+08     159600000.0    4.941000e+08     2018

In [11]:
Cleanedtoptenmovies_df= toptenmovies_df.drop("domestic-pct", axis=1).drop("overseas-pct", axis=1) 

In [12]:
Cleanedtoptenmovies_df

bo_year_rank                                       title studio  \
0            1                       Avengers Infinity War     BV   
1            2                               Black Panther     BV   
2            3               Jurassic World Fallen Kingdom   Uni.   
3            4                               Incredibles 2     BV   
4            5                                     Aquaman     WB   
5            6                           Bohemian Rhapsody    Fox   
6            7                                  Venom 2018   Sony   
7            8                Mission Impossible - Fallout   Par.   
8            9                                  Deadpool 2    Fox   
9           10  Fantastic Beasts The Crimes of Grindelwald     WB   

   worldwide-gross  domestic-gross  overseas-gross  bo_year  
0     2.048400e+09     678800000.0    1.369500e+09     2018  
1     1.346900e+09     700100000.0    6.469000e+08     2018  
2     1.309500e+09     417700000.0    8.918000e+08     2018  
3     1.242800e+09     608600000.0    6.342000e+08     2018  
4     1.147700e+09     335100000.0    8.126000e+08     2018  
5     9.026000e+08     216400000.0    6.862000e+08     2018  
6     8.550000e+08     213500000.0    6.415000e+08     2018  
7     7.911000e+08     220200000.0    5.710000e+08     2018  
8     7.789000e+08     318500000.0    4.605000e+08     2018  
9     6.537000e+08     159600000.0    4.941000e+08     2018

  if x is >= 1000000000 then convert to "1.xx billion"  to make more readable we are going from float to string 
  else convert to xxxmillion 
  
  Next steps for arjun figure out function above then put finished dataframe into mongodbcollection 
  

In [13]:
# Connect to mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Connect to Top 10 database
db = client.top_10_db

# If collection books exists, drop it so the new top 10 information will replace it
db.movies.drop()

#Create new empty books collection
movies = db.movies

# Insert top 10 books/movies/music into database
data = Cleanedtoptenmovies_df.to_dict(orient='records')
db.movies.insert_many(data)